In [12]:
# Importing packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

# Check device to run the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [13]:
# Data preprocessing
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  
    transforms.RandomCrop(32, padding=4),  
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  
])

# Load datasets
trainset = torchvision.datasets.CIFAR100(root="/home/BTECH_7TH_SEM/Desktop", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root="/home/BTECH_7TH_SEM/Desktop", train=False, download=True, transform=transform)

# Create data loaders
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


100.0%


In [ ]:
# Define the CNN model for CIFAR-100
class CIFAR100_CNN(nn.Module):
    def __init__(self):
        super(CIFAR100_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        
        self.fc1 = nn.Linear(256 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 100)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        x = x.view(-1, 256 * 4 * 4)  # flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

model = CIFAR100_CNN().to(device)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Model training and evaluation loops
def train_model(model, trainloader, testloader, epochs=100):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/{epochs}"):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluation after each loop (validation)
        acc = evaluate_model(model, testloader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(trainloader):.4f}, Test Acc: {acc:.2f}%")

# Evaluation function
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

train_model(model, trainloader, testloader, epochs=100)


Epoch 1/100: 100%|██████████| 391/391 [00:02<00:00, 176.95it/s]


Epoch [1/100], Loss: 2.6291, Test Acc: 35.88%


Epoch 2/100: 100%|██████████| 391/391 [00:02<00:00, 176.48it/s]


Epoch [2/100], Loss: 2.5314, Test Acc: 38.77%


Epoch 3/100: 100%|██████████| 391/391 [00:02<00:00, 177.71it/s]


...

Epoch [98/100], Loss: 1.0081, Test Acc: 59.39%


Epoch 99/100: 100%|██████████| 391/391 [00:02<00:00, 179.24it/s]


Epoch [99/100], Loss: 1.0131, Test Acc: 58.96%


Epoch 100/100: 100%|██████████| 391/391 [00:02<00:00, 177.77it/s]


Epoch [100/100], Loss: 1.0037, Test Acc: 58.32%
